<a href="https://colab.research.google.com/github/tarakantaacharya/TTS_ODIA/blob/main/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from transformers import AutoProcessor, AutoModelForPreTraining


# tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")

# model = AutoModelForPreTraining.from_pretrained("facebook/mms-1b")

In [13]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForTextToWaveform
import torch

# Load model in int8 quantization without bitsandbytes (works on CPU)
model = AutoModelForTextToWaveform.from_pretrained(
    "facebook/mms-tts-ory",
    torch_dtype=torch.float16,  # Use int8 quantization (CPU-compatible)
    device_map="cpu"  # Force CPU usage
)

print("Model loaded with int8 quantization on CPU successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loaded with int8 quantization on CPU successfully!


In [ ]:
model

VitsModel(
  (text_encoder): VitsTextEncoder(
    (embed_tokens): Embedding(76, 192)
    (encoder): VitsEncoder(
      (layers): ModuleList(
        (0-5): 6 x VitsEncoderLayer(
          (attention): VitsAttention(
            (k_proj): Linear(in_features=192, out_features=192, bias=True)
            (v_proj): Linear(in_features=192, out_features=192, bias=True)
            (q_proj): Linear(in_features=192, out_features=192, bias=True)
            (out_proj): Linear(in_features=192, out_features=192, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (layer_norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (feed_forward): VitsFeedForward(
            (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
            (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
            (dropout): Dropout(p=0.1, inplace=False)
            (act_fn): ReLU()
          )
          (final_layer_norm): LayerNorm((192,), eps=1e-05, eleme

In [55]:
import pickle
import pandas as pd

try:
    df = pd.read_pickle("new_preprocess_data.pkl")  # Load
except pickle.UnpicklingError as e:
    print(f"Error loading pickle file: {e}")
    print("The pickle file might be corrupted or incomplete.")
    print("Try regenerating the pickle file if possible.")
    # Add any error handling or data recovery logic here

Error loading pickle file: pickle data was truncated
The pickle file might be corrupted or incomplete.
Try regenerating the pickle file if possible.


In [50]:
df['padded_text'] = df['padded_text'].apply(lambda x: np.array(x))

In [51]:
df['padded_text'] = df['padded_text'].apply(lambda x: x.astype(float))

AttributeError: 'str' object has no attribute 'astype'

In [42]:
df = df.rename(columns={'Unnamed: 0': 'Index'})

In [43]:
df.columns

Index(['Index', 'padded_audio', 'padded_text'], dtype='object')

In [44]:
print(df.iloc[0])

Index                                                           0
padded_audio                              [0. 0. 0. ... 0. 0. 0.]
padded_text     [2.0000e+00 4.1000e+01 6.2688e+04 1.0000e+00 4...
Name: 0, dtype: object


In [14]:
import numpy as np

In [49]:
type(df['padded_audio'].iloc[1])

str

In [52]:
type(df['padded_text'].iloc[1])

str

In [40]:
import numpy as np

# Assuming 'padded_audio' column contains strings representing NumPy arrays
df['padded_audio'] = df['padded_audio'].apply(lambda x: np.fromstring(x[1:-1], sep=' '))

# Function to convert string representation of list to list of numbers
def str_to_list(text):
    # Remove brackets and split the string by commas and spaces
    elements = text[1:-1].replace(' ', '').split(',')  # Split by commas after removing spaces
    # Convert elements to numbers (handling potential floats and scientific notation)
    return [int(float(element)) for element in elements]

# Apply the function to the 'padded_text' column
df['padded_text'] = df['padded_text'].apply(str_to_list)

# Now you can access the length of the list
print(f"Audio Size:{df['padded_audio'].iloc[19].shape}")
print(f"Text Size:{len(df['padded_text'].iloc[19])}")

ValueError: could not convert string to float: '2.0000e+004.1000e+016.2688e+041.0000e+004.1000e+011.0000e+00\n6.2827e+046.2008e+041.0000e+006.2670e+046.2006e+041.0000e+00\n7.0000e+004.1000e+011.0000e+006.2828e+041.0000e+004.1000e+01\n1.0000e+006.2827e+046.2008e+041.0000e+006.2670e+041.0000e+00\n6.2007e+045.2000e+013.0000e+000.0000e+000.0000e+000.0000e+00\n0.0000e+000.0000e+000.0000e+000.0000e+000.0000e+000.0000e+00\n0.0000e+000.0000e+000.0000e+000.0000e+000.0000e+000.0000e+00\n0.0000e+000.0000e+000.0000e+000.0000e+000.0000e+000.0000e+00\n0.0000e+000.0000e+000.0000e+000.0000e+000.0000e+000.0000e+00\n0.0000e+000.0000e+000.0000e+000.0000e+000.0000e+000.0000e+00\n0.0000e+000.0000e+000.0000e+000.0000e+000.0000e+000.0000e+00'

In [ ]:
print(len(df))

1100


In [24]:
print(df.head())

   Index padded_audio                                        padded_text
0      0           []  [2.0000e+00 4.1000e+01 6.2688e+04 1.0000e+00 4...
1      1           []  [2.0000e+00 4.1000e+01 6.2688e+04 1.0000e+00 4...
2      2           []  [2.0000e+00 4.1000e+01 6.2688e+04 1.0000e+00 4...
3      3           []  [2.0000e+00 4.1000e+01 6.2688e+04 1.0000e+00 4...
4      4           []  [2.0000e+00 4.1000e+01 6.2688e+04 1.0000e+00 4...


In [ ]:
import pandas as pd
from datasets import Dataset

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df.drop(columns=['Index']))

# Check if conversion was successful
print(dataset)

Dataset({
    features: ['audio', 'text'],
    num_rows: 1100
})


In [ ]:
# Function to check the shape of audio arrays
def check_audio_shape(df):
    # Create a dictionary to store the counts of each shape
    shape_counts = {}

    for audio in df['padded_audio']:
        # Get the shape of the audio (assuming each audio is a numpy array or list)
        shape = len(audio)

        # Update the count for the shape
        if shape in shape_counts:
            shape_counts[shape] += 1
        else:
            shape_counts[shape] = 1

    return shape_counts

# Get the shape counts
shape_counts = check_audio_shape(df)

# Print the result
for shape, count in shape_counts.items():
    print(f"Audio with shape {shape}: {count} files")

NameError: name 'df' is not defined

In [ ]:
from transformers import Trainer, TrainingArguments

# Training arguments with remove_unused_columns set to False
training_args = TrainingArguments(
    output_dir="./results",  # Output directory for model checkpoints
    evaluation_strategy="steps",
    num_train_epochs=3,  # Number of epochs for fine-tuning
    per_device_train_batch_size=4,  # Use smaller batch size for CPU
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    remove_unused_columns=False  # Ignore unused columns in dataset
)

# Trainer setup for fine-tuning
trainer = Trainer(
    model=model,  # Quantized model
    args=training_args,
    train_dataset=train_dataset,  # Your training data
    eval_dataset=test_dataset,  # Your test data
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: acharyatarakanta2002 (acharyatarakanta2002-gayatri-vidya-parishad-college-of-e) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


ValueError: expected sequence of length 49421 at dim 1 (got 84428)